In [1]:
from functools import lru_cache
import logging
from threading import Thread
import time

## Review closure: function has function argument
+ Thread(target=f)
+ model.compile(loss=f)
+ QPushButton(connect=f)
+ pd.apply(f)

In [2]:
# no args
def hello():
    print("hello")
t = Thread(target=hello)
t.start()
t.join()

# args: way 1
def wrap(name):
    def hello2():
        print(f"hello {name}")
    return hello2
t = Thread(target=wrap("James"))
t.start()
t.join()

# args: way 2
def hello2(name):
    print(f"hello {name}")
t = Thread(target=lambda :hello2("James"))
t.start()
t.join()

hello
hello James
hello James


## Raw issue

In [3]:
def get_twice_and_print(x: int):
    print(x * 2)
    return x * 2
    
get_twice_and_print(3)

6


6

In [4]:
# must modify calling code so ugly
def wrap(f):
    logging.warning(f"{f.__name__} is running")
    return f

def get_twice_and_print(x: int):
    print(x * 2)
    return x * 2
    
wrap(get_twice_and_print)(3)

6


6

## Decorator: add syntax without modify function and calling code e.g. logging, execution time

#### 1. before function + without wrapping argument

In [5]:
# wrap is a "function in function out" function and can be decorator syntax
def wrap(f):
    logging.warning(f"{f.__name__} is running")
    return f

@wrap
def get_twice_and_print(x: int):
    print(x * 2)
    return x * 2

get_twice_and_print(3)

6


6

#### 2. before function + with wrapping argument

In [6]:
# wrap_args is a "arg in function out" function and can be decorator syntax
def wrap_args(msg="hi"):
    def wrap(f):
        logging.warning(f"{f.__name__} is running. {msg}")
        return f
    return wrap

@wrap_args(msg="x")
def get_twice_and_print(x: int):
    print(x * 2)
    return x * 2

get_twice_and_print(3)

6


6

#### 3. before and after function + without wrapping argument

In [7]:
def wrap(f):
    def time_count(*args, **kwargs):
        start = time.time()
        f(*args, **kwargs)
        print(time.time() - start)
        return time.time() - start
    return time_count

@wrap
def get_twice_and_print(x: int):
    print(x * 2)
    return x * 2

get_twice_and_print(3)

6
5.53131103515625e-05


6.818771362304688e-05

#### 4. before and after function + with wrapping argument

In [8]:
def wrap_args(msg="hi"):
    def wrap(f):
        def time_count(*args, **kwargs):
            start = time.time()
            f(*args, **kwargs)
            print(time.time() - start, msg)
            return time.time() - start
        return time_count
    return wrap

@wrap_args(msg="x")
def get_twice_and_print(x: int):
    print(x * 2)
    return x * 2

get_twice_and_print(3)

6
6.580352783203125e-05 x


9.369850158691406e-05

## Useful decorator: functool cache

In [9]:
def fib(n: int):
    return fib(n-1) + fib(n-2) if n > 1 else n

for i in range(3):
    start = time.time()
    print(fib(30))
    print(time.time() - start)

832040
0.1944746971130371
832040
0.18289804458618164
832040
0.1871659755706787


In [10]:
@lru_cache(maxsize=None)
def cache_fib(n: int):
    return fib(n-1) + fib(n-2) if n > 1 else n

for i in range(3):
    start = time.time()
    print(cache_fib(30))
    print(time.time() - start)

832040
0.1826777458190918
832040
6.198883056640625e-06
832040
5.4836273193359375e-06
